> *Hi Kaggle community.
> I just found this library and I'm excited by what it means for ML.
This competition ran on Kaggle, hence sharing it here made sense
> Publishing this here for research and reference only.I do not own the library or the notebook. If you like this, please check the original website out(mentioned in credits at the end).If the original contributors are reading this, please let me know, I'll add you as collaborators.
> Thanks
> -Kriti

<Title>  Covid-19 RNA vaccine degradation data set </Title>
description: Using the QLattice to predict reactivity for the structure of mRNA vaccines


by: [Chris Cave, Kevin Broløs and Sam Demharter](https://www.abzu.ai/team)

Feyn version: 2.0+

In this tutorial we are going to go through a typical QLattice workflow. We perform an analysis on the [OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction](https://www.kaggle.com/c/stanford-covid-vaccine) dataset.

The raw dataset consists of 2400 mRNA samples. Each mRNA consists of 107 nucleotides and various measurments were performed on the first 68 nucleotides. These measurements consisted of reactivity, degradation at pH10 with and without magnesium, and degradation at $50^o C$ with and without magnesium.

Some of the Covid-19 vaccines are mRNA based. However due to the unstable nature of mRNA they must be refrigerated in extreme conditions. What this means is that distribution of the vaccine can be problematic. 

The aim of this tutorial is to gain insights into the stability of general mRNA samples with the potential to apply it to Covid-19 vaccine candidates.

<h1> Installation </h1>

In [ ]:
!pip install feyn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import feyn
from sklearn.model_selection import train_test_split

from IPython.display import display

## Inspecting the raw data

Here we can see that the data we have is the sequence and the predicted structure and loop type of each base in the RNA. The feature `reactivity` measures the degradation at each base. The higher the `reactivity` the more likely the RNA is to degrade at that base.

In [ ]:
data = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)

In [ ]:
data = data.drop('index', axis=1)

In [ ]:
data.query('SN_filter == 1', inplace=True)

length = len(data.iloc[0]['reactivity'])
first_68 = data['structure'].apply(lambda x : x[0: length])

# Remove sequences that only contain “.” i.e. unpaired bases
idx_all_dots = [i for i in first_68.index if first_68[i].count('.') == length]
data = data.drop(idx_all_dots)

In [ ]:
data.head()

There's a column in this data set called `SN_filter`. This is the signal to noise filter capturing which RNA molecules that passed the evaluation criteria defined by the Stanford researchers. This means we will drop the rows with `SN_filter == 0`

There are some RNAs that have quite a large amount of noise which is filtered by the `SN_filter`. 

There are also RNAs that are all "." for the first 68 bases. During analysis, we've removed these sequences, as they were hard to capture since they have no variation in predicted structure, but a lot of variation in reactivity. This could be due to it having a very complex structure that is not represented here, or the predicted loop type being incorrect, or something entirely different.

## Preparing the sequences for the `QLattice`

Here, we prepare the sequences for the `QLattice`, by expanding the sequences into individual samples for each nucleobase. This means, that each sample consists of one nucleobase (`base`), and a predicted loop type (`loop`).

In order to maintain information about its position and relevance in the structure, we add some features about its surrounding neighbours. We call these **motifs**.

The motifs are defined by a left-side (5') and right-side (3') window of the two neighbouring bases. We then do the same, but for the loop type.

In [ ]:
end_pos = len(data.loc[0, 'predicted_loop_type'])

RNA_idx = [j for j in data.index for i in range(0, end_pos)]
pos_idx = [i for j in data.index for i in range(0, end_pos)]

loop_exp = data['predicted_loop_type'].apply(lambda x : list(x)).agg(sum)
base_exp = data['sequence'].apply(lambda x : list(x)).agg(sum)

exp_df = pd.DataFrame({'loop' : loop_exp, 'base': base_exp, 'RNA_idx' : RNA_idx, 'pos_idx' : pos_idx})

In [ ]:
react_len = len(data.iloc[0].reactivity)

In [ ]:
df = exp_df[exp_df.pos_idx < react_len]
df = df[df.pos_idx >= 5]

df['reactivity'] = df.apply(lambda row: data.loc[row.RNA_idx].reactivity[row.pos_idx], axis=1)

df['sequence'] = data.loc[df.RNA_idx].set_index(df.index).sequence
df['base_left_motif'] = df.apply(lambda x: x['sequence'][:x.pos_idx][-2:], axis=1)
df['base_right_motif'] = df.apply(lambda x: x['sequence'][x.pos_idx + 1:][:2], axis=1)

df['loop_type'] = data.loc[df.RNA_idx].set_index(df.index).predicted_loop_type
df['loop_left_motif'] = df.apply(lambda x: x['loop_type'][:x.pos_idx][-2:], axis=1)
df['loop_right_motif'] = df.apply(lambda x: x['loop_type'][x.pos_idx + 1:][:2], axis=1)

## Train, validation and holdout split

We split our train/validation/holdout split according to the sequence it belongs to. This is to make sure we don't contaminate our validation and holdout sets with samples from the same sequences we have trained on. We've captured the original sequences in the column `RNA_idx` to make this splitting easier. We have some other meta columns like this, that we'll remove prior to training.

In [ ]:
train_idx, remain_idx = train_test_split(list(data.index),train_size = 0.5, random_state = 42)
valid_idx, holdout_idx = train_test_split(remain_idx,train_size = 0.5, random_state = 42)

train = df.query('RNA_idx == @train_idx')
valid = df.query('RNA_idx == @valid_idx')
holdout = df.query('RNA_idx == @holdout_idx')

Here is our training set. We've expanded the previous data set so we have a feature for `base`, `predicted loop type`, `base_left_motif`, `base_right_motif`, `left_loop_motif` and `right_loop_motif`

In [ ]:
train.head()

# Approach 1: Training a QGraph using all the features

The first approach is to connect to the `QLattice`, and just fit a graph using all of the available features.
For this training, we've picked a max depth that is sure to be able to fit all the features - but still free enough to make more complex graphs if it should want to.

In [ ]:
# Connecting to the QLattice
ql = feyn.connect_qlattice()

# Seeding the QLattice for reproducible results
ql.reset(42)

In [ ]:
# Output variable
output = 'reactivity'

# Declaring features
features = ['base', 'loop', 'base_left_motif', 'base_right_motif', 'loop_left_motif', 'loop_right_motif']

# Declaring categorical features
stypes = {}
for f in features:
    if train[f].dtype =='object':
        stypes[f] = 'c'

In order to penalize more complex graphs that don't necessarily add enough utility compared to their simpler counterparts, we use [**AIC**](https://en.wikipedia.org/wiki/Akaike_information_criterion) as a selection criterion prior to updating the `QLattice` with the best graphs.

This avoids redundancies in your graphs, so that each interaction is potentially contributing something useful whenever possible.

This is a **regression** task, which is default for `auto_run`

In [ ]:
models = ql.auto_run(train[features+[output]], output, stypes=stypes, criterion='aic')

In [ ]:
model_base = models[0]

In [ ]:
model_base.plot(train)

We've supplied a random seed to this `QLattice`, but depending on version you might still be experiencing different results, so keep that in mind as you try this for yourself.

Looking at the plot above it appears that one of them doesn't add a lot of signal to the model - notice how the `loop_left_motif` into the next interaction of the graph doesn't seem to increase the signal by much. This is more obvious when compared to the added signal from combining the other features in the graph.

## Let's constrain the models

The previous summary plot tells us is that we should try to restrict the graph a bit more to force the `QLattice` to choose the best features, as it won't have room to use them all.

We can achieve this constraint by setting the max complexity to 7, which would allow for the individual models to only contain maximum 4 features.

In [ ]:
ql.reset(42)

In [ ]:
models = ql.auto_run(train[features+[output]], output, stypes=stypes, max_complexity=7, criterion='aic')

In [ ]:
model_constrained = models[0]

Taking a look at the train and validation sets:

In [ ]:
print('The base model (unconstrained):')
display(model_base.plot(train, valid))
print("The constrained model (max complexity = 7):")
display(model_constrained.plot(train, valid))

Both of these graphs actually appear to generalize quite well. However, it also confirms our suspicions that the left motifs are less important.

## What's next?

So we seem to have a stable model with only four features.

Now it'd be interesting to see just how far down we can carve this model until it falls apart.

It doesn't necessarily make sense to constrain the complexity of the graph further, so instead, we'll try to get rid of the features that seem to contribute the least.

In the above summary plot, it would appear that the `loop_right_motif` contributes the least to the growth of signal in the model, so let's try to remove it

In [ ]:
ql.reset(42)

features = ['base', 'loop', 'base_right_motif']

In [ ]:
models = ql.auto_run(train[features+[output]], output, stypes=stypes, max_complexity=7, criterion='aic')

In [ ]:
model_three_features = models[0]

In [ ]:
print('The constrained model:')
display(model_constrained.plot(train, valid))
print("The three feature model:")
display(model_three_features.plot(train, valid))

We're very close here. It did get a little worse - but if we want to tend towards simpler, more interpretble models, the second model here is definitely what we should go for.

It also already provides so much signal, that we can expect it to explain most of the behaviour we are able to describe using these features.

That said - we might now already also see that the base doesn't supply much information, so let's try to remove it and see how dramatic of a difference that makes.

In [ ]:
ql.reset(42)

features = ['loop', 'base_right_motif']

In [ ]:
# Note we're reducing to max complexity of 3 for two features.
models = ql.auto_run(train[features+[output]], output, stypes=stypes, max_complexity=3, criterion='aic')

In [ ]:
model_two_features = models[0]

In [ ]:
print("The three feature model:")
display(model_three_features.plot(train, valid))
print("The two feature model:")
display(model_two_features.plot(train, valid))

We see a bit of a drop but that's also in line with our expectations.
Still, we seem to be getting closer to the essence of the model here. The `loop` and the `base_right_motif`together.

We could reduce further down to just the loop - but without feature interactions it's fairly obvious that the performance would reduce to just the explanative power of the loop feature.

### Let's look at an interesting sequence

Now we have a few models we can take a look at an intersting sequence and try to map out what kind of tradeoffs we get over different complexities.

<img alt="Secondary structure of sequence 642" src="data:image/svg+xml,%3Csvg%20xmlns%3Axlink%3D%22http%3A%2F%2Fwww.w3.org%2F1999%2Fxlink%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20width%3D%22978%22%20height%3D%22648%22%20id%3D%22plotting-area%22%20style%3D%22height%3A%20648px%3B%20width%3A%20978px%3B%22%3E%3Cstyle%3Esvg%7Bdisplay%3Ablock%3Bmin-width%3A100%25%3Bwidth%3A100%25%3Bmin-height%3A100%25%3B%7Dcircle.node%7Bstroke%3A%23ccc%3Bstroke-width%3A1px%3Bopacity%3A1%3Bfill%3Awhite%3B%7Dcircle.node.label%7Bstroke%3Atransparent%3Bstroke-width%3A0%3Bfill%3Awhite%3B%7Dcircle.outline_node%7Bstroke-width%3A1px%3Bfill%3Ared%3B%7Dcircle.protein%7Bfill%3Agray%3Bfill-opacity%3A0.5%3Bstroke-width%3A4%3B%7Dcircle.hidden_outline%7Bstroke-width%3A0px%3B%7Dline.link%7Bstroke%3A%23999%3Bstroke-opacity%3A0.8%3Bstroke-width%3A2%3B%7Dline.basepair%7Bstroke%3Ared%3B%7Dline.intermolecule%7Bstroke%3Ablue%3B%7Dline.chain_chain%7Bstroke-dasharray%3A3%2C3%3B%7Dline.fake%7Bstroke%3Agreen%3B%7D.transparent%7Bfill%3Atransparent%3Bstroke-width%3A0%3Bstroke-opacity%3A0%3Bopacity%3A0%3Bvisibility%3Ahidden%3B%7D.drag_line%7Bstroke%3A%23999%3Bstroke-width%3A2%3Bpointer-events%3Anone%3B%7D.drag_line_hidden%7Bstroke%3A%23999%3Bstroke-width%3A0%3Bpointer-events%3Anone%3B%7D.d3-tip%7Bline-height%3A1%3Bfont-weight%3Abold%3Bpadding%3A6px%3Bbackground%3Argba%280%2C0%2C0%2C0.6%29%3Bcolor%3A%23fff%3Bborder-radius%3A4px%3Bpointer-events%3Anone%3B%7Dtext.node-label%7Bfont-weight%3Abold%3Bfont-family%3ATahoma%2CGeneva%2Csans-serif%3Bcolor%3Argb%28100%2C100%2C100%29%3Bpointer-events%3Anone%3B%7Dtext%7Bpointer-events%3Anone%3B%7Dg.gnode%7B%7Dcircle.outline_node.selected%7Bvisibility%3Avisible%3B%7Dcircle.outline_node%7Bvisibility%3Ahidden%3B%7D.brush.extent%7Bfill-opacity%3A.1%3Bstroke%3A%23fff%3Bshape-rendering%3AcrispEdges%3B%7D.noselect%7B-webkit-touch-callout%3Anone%3B-webkit-user-select%3Anone%3B-khtml-user-select%3Anone%3B-moz-user-select%3Anone%3B-ms-user-select%3Anone%3Buser-select%3Anone%3B%7D%3C%2Fstyle%3E%3Cg%3E%3Crect%20width%3D%22978%22%20height%3D%22648%22%20fill%3D%22white%22%20stroke%3D%22transparent%22%20stroke-width%3D%221%22%20id%3D%22zrect%22%2F%3E%3Cg%20transform%3D%22translate%28146.05905064022465%2C119.15089623233095%29%20scale%281.4525339119977019%29%22%3E%3Cg%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22261.33022977467175%22%20y1%3D%22205.7179569780551%22%20x2%3D%22269.92134485342103%22%20y2%3D%22193.28922349547497%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E09d63960-6fbc-47ee-b0e9-002d35dac985%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22269.92134485342103%22%20y1%3D%22193.28922349547497%22%20x2%3D%22280.8035276815519%22%20y2%3D%22182.82036160752085%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E65caab08-1c15-45cb-9b0c-d5281fb4ceea%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22280.8035276815519%22%20y1%3D%22182.82036160752085%22%20x2%3D%22293.74361715181374%22%20y2%3D%22174.6721829685837%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E277b6228-45ff-40df-835b-c2c2229901fd%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22293.74361715181374%22%20y1%3D%22174.6721829685837%22%20x2%3D%22308.00469861783876%22%20y2%3D%22169.2034001188121%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E21d35cf6-f7de-4720-bf60-4ebde1111dc0%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22323.57559352370595%22%20y1%3D%22167.32372579057125%22%20x2%3D%22338.08486636273875%22%20y2%3D%22165.38440186906934%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3f51384a-5e48-4f09-a2f3-f914efb5b31c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22308.00469861783876%22%20y1%3D%22169.2034001188121%22%20x2%3D%22323.57559352370595%22%20y2%3D%22167.32372579057125%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed03f66dc-5f59-4fba-b32b-3dcb1d1d522b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22318.66952059531167%22%20y1%3D%22154.31270456052513%22%20x2%3D%22330.9383778512908%22%20y2%3D%22147.66942928102875%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eff0a7c4c-3e10-4a24-a31e-22e11af2b4d7%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22323.57559352370595%22%20y1%3D%22167.32372579057125%22%20x2%3D%22318.66952059531167%22%20y2%3D%22154.31270456052513%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E096bee42-c078-46b7-bdff-462b13544b4d%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22309.7076251862503%22%20y1%3D%22141.75325952564467%22%20x2%3D%22321.4174514972199%22%20y2%3D%22132.74189680087258%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7b297d4f-627a-4c91-918b-05cb6b95bd50%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22318.66952059531167%22%20y1%3D%22154.31270456052513%22%20x2%3D%22309.7076251862503%22%20y2%3D%22141.75325952564467%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E41cdcefd-9916-420b-b837-9ba6202ff63e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22309.7076251862503%22%20y1%3D%22141.75325952564467%22%20x2%3D%22294.3273054775648%22%20y2%3D%22143.39093063047775%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3f4131d3-48ec-4d37-b255-66ce4b8c97b9%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22294.3273054775648%22%20y1%3D%22143.39093063047775%22%20x2%3D%22280.43262811992%22%20y2%3D%22137.5041177908933%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E0c945eda-53f2-4060-8e67-dfd46758129e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22280.43262811992%22%20y1%3D%22137.5041177908933%22%20x2%3D%22271.02504300854974%22%20y2%3D%22125.95130834466995%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ec5450bf9-d799-4deb-a91d-1cfb414bb7f2%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22268.3305683413489%22%20y1%3D%22110.70828701440702%22%20x2%3D%22274.89626562404305%22%20y2%3D%2297.44929927008413%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed41dfdd7-4cc1-4300-a00b-7e806373a8ae%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22271.02504300854974%22%20y1%3D%22125.95130834466995%22%20x2%3D%22268.3305683413489%22%20y2%3D%22110.70828701440702%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E20c5ca3e-11f3-4d03-9392-e90a2fe05817%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22253.59327465157804%22%20y1%3D%22102.67471388450429%22%20x2%3D%22261.3567586734708%22%20y2%3D%2290.22984782122066%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E45b47e50-f32d-45b4-aac7-4628c7a40b6b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22268.3305683413489%22%20y1%3D%22110.70828701440702%22%20x2%3D%22253.59327465157804%22%20y2%3D%22102.67471388450429%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E43a0ee0c-8f96-4cf2-aef9-e7940640fac1%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22237.26378383911214%22%20y1%3D%2297.76213878304395%22%20x2%3D%22237.8401227166589%22%20y2%3D%2283.0285421787688%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E49de461c-3de3-4089-a543-a1cab5287cc8%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22253.59327465157804%22%20y1%3D%22102.67471388450429%22%20x2%3D%22237.26378383911214%22%20y2%3D%2297.76213878304395%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Edc2a91d4-2046-4987-935d-dd8330048910%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22220.20625722162484%22%20y1%3D%2295.44446768147112%22%20x2%3D%22223.07197284879553%22%20y2%3D%2281.28316697289921%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eee17c1ba-7e01-4c31-8bb6-b8d5267b3f98%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22237.26378383911214%22%20y1%3D%2297.76213878304395%22%20x2%3D%22220.20625722162484%22%20y2%3D%2295.44446768147112%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E40a20ec2-0ff2-4f35-a058-3952ec472cbb%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22203.7082151975234%22%20y1%3D%2290.88999744543656%22%20x2%3D%22208.51615300181905%22%20y2%3D%2277.22113265498095%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E22b6b0e9-e599-4428-86bd-ba5a6e134530%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22220.20625722162484%22%20y1%3D%2295.44446768147112%22%20x2%3D%22203.7082151975234%22%20y2%3D%2290.88999744543656%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Edc176cad-dbab-4300-8ed7-0cce786c9255%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22188.0665623167126%22%20y1%3D%2284.49131320760816%22%20x2%3D%22194.64823371633932%22%20y2%3D%2271.22539090186221%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E2d0141de-defb-4905-ba73-09cdca13708c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22203.7082151975234%22%20y1%3D%2290.88999744543656%22%20x2%3D%22188.0665623167126%22%20y2%3D%2284.49131320760816%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E8ba72baf-df63-4668-b81c-caa140634f1e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22188.0665623167126%22%20y1%3D%2284.49131320760816%22%20x2%3D%22173.88467720906812%22%20y2%3D%2290.83474235261048%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E37f1bc34-cf8a-4536-abd9-aabd3b1db14d%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22173.88467720906812%22%20y1%3D%2290.83474235261048%22%20x2%3D%22159.23052399432208%22%20y2%3D%2287.61903658978987%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eaa4cf16e-fbe8-495e-b7be-11018584248a%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22159.23052399432208%22%20y1%3D%2287.61903658978987%22%20x2%3D%22148.917393194143%22%20y2%3D%2276.75091789712536%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ef96737cf-f27c-49de-a8da-dafb50153bdf%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22146.8605766869757%22%20y1%3D%2261.38899007091598%22%20x2%3D%22154.62161036492927%22%20y2%3D%2248.75956934937379%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E2d393664-28a2-45af-a145-f98c4d113f0c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22148.917393194143%22%20y1%3D%2276.75091789712536%22%20x2%3D%22146.8605766869757%22%20y2%3D%2261.38899007091598%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eae5fd8c1-28a5-4957-8f32-b207b3f522f6%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22133.2393050741329%22%20y1%3D%2252.35667415380466%22%20x2%3D%22141.21854618260056%22%20y2%3D%2240.350886846244784%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E957c844d-a3b0-44fd-a292-af51523dc73c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22146.8605766869757%22%20y1%3D%2261.38899007091598%22%20x2%3D%22133.2393050741329%22%20y2%3D%2252.35667415380466%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed05f027b-ad9e-4b56-b22a-49aa59e1bdb0%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22119.85711929274632%22%20y1%3D%2243.69077441974384%22%20x2%3D%22128.09414019676365%22%20y2%3D%2231.226922368765038%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ec5f0a281-badf-4441-985d-515ab1dda074%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22133.2393050741329%22%20y1%3D%2252.35667415380466%22%20x2%3D%22119.85711929274632%22%20y2%3D%2243.69077441974384%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E51864597-4367-4e8b-8cb3-9264afa932de%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22119.85711929274632%22%20y1%3D%2243.69077441974384%22%20x2%3D%22107.22978426824206%22%20y2%3D%2252.28996924736046%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9ae5c3a5-7196-4589-9788-0dd98907f3a4%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22107.22978426824206%22%20y1%3D%2252.28996924736046%22%20x2%3D%2292.64966368977876%22%20y2%3D%2255.73696961536778%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7c23172d-c253-43d8-ae4f-eb29a65b72a6%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2292.64966368977876%22%20y1%3D%2255.73696961536778%22%20x2%3D%2277.75406024742801%22%20y2%3D%2253.739059433773846%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed3771664-1819-4e9e-82a7-7ed5f4bed113%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2277.75406024742801%22%20y1%3D%2253.739059433773846%22%20x2%3D%2264.63373271636493%22%20y2%3D%2246.51146074728539%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ee01ad320-14ba-4997-990f-378f3eb7a309%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2264.63373271636493%22%20y1%3D%2246.51146074728539%22%20x2%3D%2254.98437320541741%22%20y2%3D%2234.904227706361084%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9a6e5340-66ad-4c73-bb90-6cb01df10acf%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2254.98437320541741%22%20y1%3D%2234.904227706361084%22%20x2%3D%2250.18774085340792%22%20y2%3D%2220.73495746674428%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E715225b4-8f9f-4153-a0fc-7c70552329fc%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%2250.494542262651024%22%20y1%3D%225.371201028906774%22%20x2%3D%2256.58056588370028%22%20y2%3D%22-8.294938082925073%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eac760e85-bd27-4162-afbc-bc122a17bb09%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2250.18774085340792%22%20y1%3D%2220.73495746674428%22%20x2%3D%2250.494542262651024%22%20y2%3D%225.371201028906774%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E56a0c901-fd1f-4225-8ba3-f6dcdd714910%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%2236.0062822565994%22%20y1%3D%22-1.3295687584877784%22%20x2%3D%2241.89663720576248%22%20y2%3D%22-14.515474357989916%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eec68df32-b33f-4022-89a2-0858bfd97ac6%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2250.494542262651024%22%20y1%3D%225.371201028906774%22%20x2%3D%2236.0062822565994%22%20y2%3D%22-1.3295687584877784%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eeaf30ec1-e1c1-424b-a793-b980dfc6eace%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%2221.37894864564745%22%20y1%3D%22-7.569605192805647%22%20x2%3D%2227.51953462907894%22%20y2%3D%22-21.237525957435718%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ee2203138-927d-4575-85fc-2d5a81a8ea3c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2236.0062822565994%22%20y1%3D%22-1.3295687584877784%22%20x2%3D%2221.37894864564745%22%20y2%3D%22-7.569605192805647%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E2c032fde-eb3e-4508-bb12-c1462d0371e3%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2221.37894864564745%22%20y1%3D%22-7.569605192805647%22%20x2%3D%226.281139301491906%22%20y2%3D%22-3.7856753488644146%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E4a8e15df-9cc4-4100-a368-6284e810b035%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%226.281139301491906%22%20y1%3D%22-3.7856753488644146%22%20x2%3D%22-5.825964380613266%22%20y2%3D%22-12.948879639741032%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb62d7700-1501-4362-a2da-8858a6b35980%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22-5.825964380613266%22%20y1%3D%22-12.948879639741032%22%20x2%3D%22-6.166307094561371%22%20y2%3D%22-28.162600875898775%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E16962594-2993-4162-a5dc-8a65cf30ab8b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22-6.166307094561371%22%20y1%3D%22-28.162600875898775%22%20x2%3D%225.49192448647406%22%20y2%3D%22-37.95848201904741%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E04e2435e-0608-4cb9-83eb-7c2e96e30797%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%225.49192448647406%22%20y1%3D%22-37.95848201904741%22%20x2%3D%2220.35518375658927%22%20y2%3D%22-35.05060474965359%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E659c316b-a739-474e-9701-c27f6da8259b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%2227.51953462907894%22%20y1%3D%22-21.237525957435718%22%20x2%3D%2221.37894864564745%22%20y2%3D%22-7.569605192805647%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed83e5d83-2b31-41f0-8b5c-c55430d89d11%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2220.35518375658927%22%20y1%3D%22-35.05060474965359%22%20x2%3D%2227.51953462907894%22%20y2%3D%22-21.237525957435718%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eeeca1f97-2197-4bb5-b45b-7b30a48bbc3c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%2241.89663720576248%22%20y1%3D%22-14.515474357989916%22%20x2%3D%2236.0062822565994%22%20y2%3D%22-1.3295687584877784%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3503a37d-a6ea-4372-b5e2-ebebfabea361%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2227.51953462907894%22%20y1%3D%22-21.237525957435718%22%20x2%3D%2241.89663720576248%22%20y2%3D%22-14.515474357989916%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E67f118e2-79bf-410a-a69b-f89108bcfac7%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%2256.58056588370028%22%20y1%3D%22-8.294938082925073%22%20x2%3D%2250.494542262651024%22%20y2%3D%225.371201028906774%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E0b89ba6a-13aa-4afc-9ab1-def2aa809724%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2241.89663720576248%22%20y1%3D%22-14.515474357989916%22%20x2%3D%2256.58056588370028%22%20y2%3D%22-8.294938082925073%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E127014d9-57d4-4e02-8b0f-5c30b46c713d%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2256.58056588370028%22%20y1%3D%22-8.294938082925073%22%20x2%3D%2267.61629637228151%22%20y2%3D%22-18.995330803477085%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3025066c-cf3d-4c91-b993-d570c1581667%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2267.61629637228151%22%20y1%3D%22-18.995330803477085%22%20x2%3D%2281.24485752496938%22%20y2%3D%22-25.19996419913149%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eff0c8b90-7ecc-4e01-bdf3-31e7e0c71dd4%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2281.24485752496938%22%20y1%3D%22-25.19996419913149%22%20x2%3D%2296.32903750646497%22%20y2%3D%22-25.995926302116736%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E53a2a4d5-f72a-44c8-9d28-a5274cea50a3%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%2296.32903750646497%22%20y1%3D%22-25.995926302116736%22%20x2%3D%22110.54758163432281%22%20y2%3D%22-21.2571319747247%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E24da3f78-bf9e-4a27-945b-d95d49c3363a%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22110.54758163432281%22%20y1%3D%22-21.2571319747247%22%20x2%3D%22122.14546909718675%22%20y2%3D%22-11.761940409124042%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7e1e7f45-b4b7-4d55-80f0-9f073ff11a1c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22122.14546909718675%22%20y1%3D%22-11.761940409124042%22%20x2%3D%22129.50801838681076%22%20y2%3D%221.3943207902431431%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7a3f43f4-4593-4446-9055-a99bdbf8a7b9%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22129.50801838681076%22%20y1%3D%221.3943207902431431%22%20x2%3D%22131.5078307762571%22%20y2%3D%2216.26512688763858%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E0796b656-49e4-4e8b-870a-f792f0f77ec4%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22128.09414019676365%22%20y1%3D%2231.226922368765038%22%20x2%3D%22119.85711929274632%22%20y2%3D%2243.69077441974384%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E272ca457-6988-4e84-a2ea-b73b86c3d07f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22131.5078307762571%22%20y1%3D%2216.26512688763858%22%20x2%3D%22128.09414019676365%22%20y2%3D%2231.226922368765038%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7eda4e5c-5ac9-4987-b041-7736534cf6ca%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22141.21854618260056%22%20y1%3D%2240.350886846244784%22%20x2%3D%22133.2393050741329%22%20y2%3D%2252.35667415380466%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9bda51f1-fba3-4f17-9416-66df87c1cc13%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22128.09414019676365%22%20y1%3D%2231.226922368765038%22%20x2%3D%22141.21854618260056%22%20y2%3D%2240.350886846244784%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E74b5defe-b79a-426e-a2c5-fe40377e3b3c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22154.62161036492927%22%20y1%3D%2248.75956934937379%22%20x2%3D%22146.8605766869757%22%20y2%3D%2261.38899007091598%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E81d7c664-0128-48eb-a49c-c19c471870d7%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22141.21854618260056%22%20y1%3D%2240.350886846244784%22%20x2%3D%22154.62161036492927%22%20y2%3D%2248.75956934937379%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E6a7af551-2c2b-4532-86d4-0821f97a4c4f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22154.62161036492927%22%20y1%3D%2248.75956934937379%22%20x2%3D%22168.42593069350872%22%20y2%3D%2242.42212640371588%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9abd7f3b-02aa-4846-a1fe-6fe9c59c41df%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22168.42593069350872%22%20y1%3D%2242.42212640371588%22%20x2%3D%22183.02865017470933%22%20y2%3D%2245.09736448038358%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E8fd4e330-febd-40de-9c54-8890abd4e488%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22183.02865017470933%22%20y1%3D%2245.09736448038358%22%20x2%3D%22193.10497146266545%22%20y2%3D%2256.20934898562181%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E4cc40535-8fcb-407f-b2d0-db347abd93ba%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22194.64823371633932%22%20y1%3D%2271.22539090186221%22%20x2%3D%22188.0665623167126%22%20y2%3D%2284.49131320760816%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E0778a13c-a3aa-48e5-b94e-2de40a6ceff8%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22193.10497146266545%22%20y1%3D%2256.20934898562181%22%20x2%3D%22194.64823371633932%22%20y2%3D%2271.22539090186221%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E6ad21234-626a-49cf-9e0d-e9d7f1f1d54b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22208.51615300181905%22%20y1%3D%2277.22113265498095%22%20x2%3D%22203.7082151975234%22%20y2%3D%2290.88999744543656%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ebb4aeecf-ea59-4916-a762-59d24dfbc280%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22194.64823371633932%22%20y1%3D%2271.22539090186221%22%20x2%3D%22208.51615300181905%22%20y2%3D%2277.22113265498095%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3bfea10d-6479-44b4-8763-462c8fa7c050%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22223.07197284879553%22%20y1%3D%2281.28316697289921%22%20x2%3D%22220.20625722162484%22%20y2%3D%2295.44446768147112%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E2dde04e2-2e93-4cfa-82da-d0abebb0db32%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22208.51615300181905%22%20y1%3D%2277.22113265498095%22%20x2%3D%22223.07197284879553%22%20y2%3D%2281.28316697289921%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9b3f1a33-8456-4baa-99ac-88c00285179b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22237.8401227166589%22%20y1%3D%2283.0285421787688%22%20x2%3D%22237.26378383911214%22%20y2%3D%2297.76213878304395%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ee565b95e-3ddb-4b71-88fb-d61fbd5add3c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22223.07197284879553%22%20y1%3D%2281.28316697289921%22%20x2%3D%22237.8401227166589%22%20y2%3D%2283.0285421787688%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed7be39c1-b2ca-4530-ae82-204f79e67545%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22237.8401227166589%22%20y1%3D%2283.0285421787688%22%20x2%3D%22252.15477644938574%22%20y2%3D%2277.84775088217394%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E15013dc6-504b-4ef2-8e77-5cd5e1b8035c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22261.3567586734708%22%20y1%3D%2290.22984782122066%22%20x2%3D%22253.59327465157804%22%20y2%3D%22102.67471388450429%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E497c38cf-c9cb-4592-a1b0-e83d2044d059%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22252.15477644938574%22%20y1%3D%2277.84775088217394%22%20x2%3D%22261.3567586734708%22%20y2%3D%2290.22984782122066%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eea167eed-26be-40c5-8c9a-c181b6dd18fd%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22274.89626562404305%22%20y1%3D%2297.44929927008413%22%20x2%3D%22268.3305683413489%22%20y2%3D%22110.70828701440702%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E110923fe-66a8-4ebd-8110-3e2a9607e023%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22261.3567586734708%22%20y1%3D%2290.22984782122066%22%20x2%3D%22274.89626562404305%22%20y2%3D%2297.44929927008413%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E4dc0980e-a334-4f13-983c-2e5f6260bb18%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22274.89626562404305%22%20y1%3D%2297.44929927008413%22%20x2%3D%22286.96624396208074%22%20y2%3D%2288.23171697410628%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ebacc9c14-31e8-4231-b075-346220bedc74%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22286.96624396208074%22%20y1%3D%2288.23171697410628%22%20x2%3D%22301.6100541832667%22%20y2%3D%2285.72408910839023%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E93a88f68-372a-4456-83a7-15703f12641b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22301.6100541832667%22%20y1%3D%2285.72408910839023%22%20x2%3D%22315.5343057787833%22%20y2%3D%2291.13604854450232%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E56ac9ca1-bb6f-46c3-9b55-6ca9d9c514e7%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22315.5343057787833%22%20y1%3D%2291.13604854450232%22%20x2%3D%22324.52450555189927%22%20y2%3D%22103.32895983867078%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E59d5072e-9634-48be-8898-39a513db0e1b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22324.52450555189927%22%20y1%3D%22103.32895983867078%22%20x2%3D%22326.1950276616631%22%20y2%3D%22118.00941509813478%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ea3fb9082-0cb2-46ac-b0c5-0972883f071c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22321.4174514972199%22%20y1%3D%22132.74189680087258%22%20x2%3D%22309.7076251862503%22%20y2%3D%22141.75325952564467%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E99eabdcf-5c0c-4c90-8f3a-071163465f1e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22326.1950276616631%22%20y1%3D%22118.00941509813478%22%20x2%3D%22321.4174514972199%22%20y2%3D%22132.74189680087258%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed92271f6-13f0-468c-a279-b87d9201804f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22330.9383778512908%22%20y1%3D%22147.66942928102875%22%20x2%3D%22318.66952059531167%22%20y2%3D%22154.31270456052513%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E94757d28-57a9-4778-8508-12bbaa0944ef%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22321.4174514972199%22%20y1%3D%22132.74189680087258%22%20x2%3D%22330.9383778512908%22%20y2%3D%22147.66942928102875%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb46a8e1a-2e4b-4d6d-9e28-0415b8e741f6%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22338.08486636273875%22%20y1%3D%22165.38440186906934%22%20x2%3D%22323.57559352370595%22%20y2%3D%22167.32372579057125%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E5ff92a29-8879-44c8-8de2-5774d5f7a452%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22330.9383778512908%22%20y1%3D%22147.66942928102875%22%20x2%3D%22338.08486636273875%22%20y2%3D%22165.38440186906934%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E19dc94b0-3702-4add-9790-fee08b5053f1%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22354.76852217911056%22%20y1%3D%22169.3251818417209%22%20x2%3D%22366.8910744841675%22%20y2%3D%22177.6180403333029%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E22b5a866-0832-4fcc-82c9-9592d97b0836%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22338.08486636273875%22%20y1%3D%22165.38440186906934%22%20x2%3D%22354.76852217911056%22%20y2%3D%22169.3251818417209%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E42acfdf7-d238-43d5-bed5-82d7f4901346%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22368.9157832822891%22%20y1%3D%22156.66812096862787%22%20x2%3D%22377.15880549612746%22%20y2%3D%22168.11355191907828%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E191b86f8-c5b1-4a5c-9cf3-80ecf96c6412%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22354.76852217911056%22%20y1%3D%22169.3251818417209%22%20x2%3D%22368.9157832822891%22%20y2%3D%22156.66812096862787%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E5cc35eec-5573-4872-adfb-e00ba14bd285%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22384.01925699601077%22%20y1%3D%22147.58265387332764%22%20x2%3D%22390.65318886309734%22%20y2%3D%22160.06071408366773%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E562c060e-aea9-4742-9111-22daaf2356f8%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22368.9157832822891%22%20y1%3D%22156.66812096862787%22%20x2%3D%22384.01925699601077%22%20y2%3D%22147.58265387332764%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7db28965-ff88-4705-89a4-bb2c1656732f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22399.4126976058369%22%20y1%3D%22140.10811503343274%22%20x2%3D%22405.1769335846781%22%20y2%3D%22153.10120169467294%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb848d2ce-79e6-4fc9-88d4-f9b79a69e09f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22384.01925699601077%22%20y1%3D%22147.58265387332764%22%20x2%3D%22399.4126976058369%22%20y2%3D%22140.10811503343274%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E2e5b7f5f-ac75-476f-b4f2-54a15f327a5b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22415.02716627361883%22%20y1%3D%22133.67140771309843%22%20x2%3D%22420.05052416224527%22%20y2%3D%22147.0796197115491%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9b738c1c-ea00-46bf-8f2d-2d59fdf3decb%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22399.4126976058369%22%20y1%3D%22140.10811503343274%22%20x2%3D%22415.02716627361883%22%20y2%3D%22133.67140771309843%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E96693864-604f-4f2c-a052-c5aae8b720b8%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22430.69891142655683%22%20y1%3D%22128.1586182221843%22%20x2%3D%22435.13499930476445%22%20y2%3D%22141.91435319605782%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3a756982-3bee-4ad9-bdad-03761c360e30%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22415.02716627361883%22%20y1%3D%22133.67140771309843%22%20x2%3D%22430.69891142655683%22%20y2%3D%22128.1586182221843%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ee2ed37fb-8ef5-4fe4-93ea-6d607b008950%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22446.03379668066964%22%20y1%3D%22123.26470642992595%22%20x2%3D%22450.2672083765375%22%20y2%3D%22137.47964263807032%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E125ccf39-8936-4bae-aed5-e5b5572144e5%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22430.69891142655683%22%20y1%3D%22128.1586182221843%22%20x2%3D%22446.03379668066964%22%20y2%3D%22123.26470642992595%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E0fc6872f-04af-4d5a-9815-86366896782e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22446.03379668066964%22%20y1%3D%22123.26470642992595%22%20x2%3D%22456.75485643831473%22%20y2%3D%22112.13331778603275%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ef82679ff-a32b-4c7e-8be8-d979381cc624%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22456.75485643831473%22%20y1%3D%22112.13331778603275%22%20x2%3D%22471.27646948804073%22%20y2%3D%22115.7851490720496%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E6798e79f-9265-4383-9649-c7a49c529807%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22471.27646948804073%22%20y1%3D%22115.7851490720496%22%20x2%3D%22475.4850785880179%22%20y2%3D%22130.13922480525764%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E5727d653-769a-4088-a2f8-8efa5dfca5e8%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22475.4850785880179%22%20y1%3D%22130.13922480525764%22%20x2%3D%22465.3702123089509%22%20y2%3D%22141.00669016189443%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E99b58820-d4d5-4058-8405-a0bf59524969%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22450.2672083765375%22%20y1%3D%22137.47964263807032%22%20x2%3D%22446.03379668066964%22%20y2%3D%22123.26470642992595%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eba2334c3-89de-43e4-8403-994c4f06aa07%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22465.3702123089509%22%20y1%3D%22141.00669016189443%22%20x2%3D%22450.2672083765375%22%20y2%3D%22137.47964263807032%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E53589b7e-d3e4-435b-beed-7d34720b484a%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22435.13499930476445%22%20y1%3D%22141.91435319605782%22%20x2%3D%22430.69891142655683%22%20y2%3D%22128.1586182221843%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ec83f1a80-3f82-4c66-b065-f7d2e0f91333%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22450.2672083765375%22%20y1%3D%22137.47964263807032%22%20x2%3D%22435.13499930476445%22%20y2%3D%22141.91435319605782%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E26a2ed1d-92e4-45e1-b0a0-635e3c0f8c9f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22420.05052416224527%22%20y1%3D%22147.0796197115491%22%20x2%3D%22415.02716627361883%22%20y2%3D%22133.67140771309843%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eea0f5f86-fee7-4d69-8769-c395e7eb0e68%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22435.13499930476445%22%20y1%3D%22141.91435319605782%22%20x2%3D%22420.05052416224527%22%20y2%3D%22147.0796197115491%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E40be6239-0f60-403b-9384-48cc4739bff4%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22405.1769335846781%22%20y1%3D%22153.10120169467294%22%20x2%3D%22399.4126976058369%22%20y2%3D%22140.10811503343274%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E8b142728-7d42-4855-b6eb-ccb9398368da%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22420.05052416224527%22%20y1%3D%22147.0796197115491%22%20x2%3D%22405.1769335846781%22%20y2%3D%22153.10120169467294%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E526239cd-838d-423a-8d9b-ecfd1eb218e7%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22390.65318886309734%22%20y1%3D%22160.06071408366773%22%20x2%3D%22384.01925699601077%22%20y2%3D%22147.58265387332764%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E3a2225e4-420b-4f94-9d37-0b167084db99%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22405.1769335846781%22%20y1%3D%22153.10120169467294%22%20x2%3D%22390.65318886309734%22%20y2%3D%22160.06071408366773%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9807cc97-b493-481f-b823-243b6c61b492%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22377.15880549612746%22%20y1%3D%22168.11355191907828%22%20x2%3D%22368.9157832822891%22%20y2%3D%22156.66812096862787%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E45e301d9-c566-41a6-8f50-1ee37db72b86%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22390.65318886309734%22%20y1%3D%22160.06071408366773%22%20x2%3D%22377.15880549612746%22%20y2%3D%22168.11355191907828%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E1075ecc4-2346-4641-9999-50c761b9e1be%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20basepair%22%20x1%3D%22366.8910744841675%22%20y1%3D%22177.6180403333029%22%20x2%3D%22354.76852217911056%22%20y2%3D%22169.3251818417209%22%20link_type%3D%22basepair%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E49caccbf-0536-4ab1-88ed-f6a66596bad3%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22377.15880549612746%22%20y1%3D%22168.11355191907828%22%20x2%3D%22366.8910744841675%22%20y2%3D%22177.6180403333029%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb91e513a-c910-4a9f-926b-a78936cd50a5%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22366.8910744841675%22%20y1%3D%22177.6180403333029%22%20x2%3D%22379.8635256716172%22%20y2%3D%22186.3614092489082%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E6ebd7564-906c-41d5-920d-a4d6f06645fe%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22379.8635256716172%22%20y1%3D%22186.3614092489082%22%20x2%3D%22390.04179047506284%22%20y2%3D%22197.62478748408518%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Efd079de6-bc44-444f-8723-c721a8990fb9%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22390.04179047506284%22%20y1%3D%22197.62478748408518%22%20x2%3D%22397.86568404176774%22%20y2%3D%22210.75281579668848%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed11aed67-be5f-44c5-8329-34fb4e9fbe0c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22397.86568404176774%22%20y1%3D%22210.75281579668848%22%20x2%3D%22402.8379615708331%22%20y2%3D%22225.04323260754242%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Efab8321a-c4fd-48a0-8837-45150f2ccd0e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22402.8379615708331%22%20y1%3D%22225.04323260754242%22%20x2%3D%22404.8591390214511%22%20y2%3D%22239.9388617360107%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb3eaa8a0-1890-49a7-9aa5-9db9046dedb2%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22404.8591390214511%22%20y1%3D%22239.9388617360107%22%20x2%3D%22404.0257700986887%22%20y2%3D%22254.89230219200843%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E17ac9b2a-cee2-4364-9fef-861bf940cade%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22404.0257700986887%22%20y1%3D%22254.89230219200843%22%20x2%3D%22400.2084621543771%22%20y2%3D%22269.388240769003%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E286e9a07-f203-426e-9136-9dd1fe4e65eb%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22400.2084621543771%22%20y1%3D%22269.388240769003%22%20x2%3D%22393.60236482910176%22%20y2%3D%22282.7906104059658%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9f435141-77d8-4709-83ed-5b12445d5b78%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22393.60236482910176%22%20y1%3D%22282.7906104059658%22%20x2%3D%22384.5341062238146%22%20y2%3D%22294.72053130967106%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E41688ce3-721d-4c68-941a-9ae923c3d96f%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22384.5341062238146%22%20y1%3D%22294.72053130967106%22%20x2%3D%22373.3472274621469%22%20y2%3D%22304.68393590894107%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb8aecfec-2341-4fd6-827f-496dcc21881c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22373.3472274621469%22%20y1%3D%22304.68393590894107%22%20x2%3D%22360.4011347697722%22%20y2%3D%22312.31147081991884%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E140fbd33-1fe8-4aca-a1fb-644bc47704e4%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22360.4011347697722%22%20y1%3D%22312.31147081991884%22%20x2%3D%22346.24857705373563%22%20y2%3D%22317.2339439428048%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eedbd55a2-dab4-438b-932a-0abffa3a5503%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22346.24857705373563%22%20y1%3D%22317.2339439428048%22%20x2%3D%22331.3507385824045%22%20y2%3D%22319.69363260866135%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E32e20bd9-7876-468c-b3a1-2aec6ed153ac%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22331.3507385824045%22%20y1%3D%22319.69363260866135%22%20x2%3D%22316.39399554833847%22%20y2%3D%22317.34088856556275%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ef75b2662-94c0-4bfb-b85b-394cb8d15a0a%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22316.39399554833847%22%20y1%3D%22317.34088856556275%22%20x2%3D%22301.9559657413121%22%20y2%3D%22313.47119773208703%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E5775d04a-afe8-4672-8ce4-a29c110f291b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22301.9559657413121%22%20y1%3D%22313.47119773208703%22%20x2%3D%22287.810092752428%22%20y2%3D%22308.1462399834821%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E1df530ea-7998-4e6a-b599-690095a60637%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22287.810092752428%22%20y1%3D%22308.1462399834821%22%20x2%3D%22276.0605843669028%22%20y2%3D%22298.65887104079934%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E93d6d247-b93b-408b-8f89-51cbe8494970%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22276.0605843669028%22%20y1%3D%22298.65887104079934%22%20x2%3D%22266.20438425261375%22%20y2%3D%22287.38633987887266%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ed68e8d53-5084-4305-a9f4-d939268aedbd%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22266.20438425261375%22%20y1%3D%22287.38633987887266%22%20x2%3D%22258.6842665238248%22%20y2%3D%22274.3830007132106%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E1ff10bfb-7b3f-4a9c-a8ca-8766cb60bce5%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22258.6842665238248%22%20y1%3D%22274.3830007132106%22%20x2%3D%22253.85197554987042%22%20y2%3D%22260.185323691355%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E4534ea81-fc02-4dd8-b854-1b785ed1d49b%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20backbone%22%20x1%3D%22253.85197554987042%22%20y1%3D%22260.185323691355%22%20x2%3D%22251.8589387621977%22%20y2%3D%22245.31339242721728%22%20link_type%3D%22backbone%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E9c272595-4439-48c0-85ec-39e14590c38e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22280.43262811992%22%20y1%3D%22137.5041177908933%22%20x2%3D%22270.1471072104067%22%20y2%3D%22149.01711800020846%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E1c7076ff-b8e0-4bdf-85de-75023abbfa23%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22148.917393194143%22%20y1%3D%2276.75091789712536%22%20x2%3D%22140.5233287677928%22%20y2%3D%2289.78973909361002%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E5b9f3e00-5464-4b0e-88eb-5bc07e82faff%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%2250.494542262651024%22%20y1%3D%225.371201028906774%22%20x2%3D%2239.21335174637153%22%20y2%3D%2216.04793652179012%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Ea52dafa5-5d67-4568-a54f-b5d3e9891d2c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%2256.58056588370028%22%20y1%3D%22-8.294938082925073%22%20x2%3D%2256.68883556563214%22%20y2%3D%22-23.818584916003246%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E70402e7e-ea39-45b7-9307-a7910ba224d8%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22154.62161036492927%22%20y1%3D%2248.75956934937379%22%20x2%3D%22159.8504411174667%22%20y2%3D%2234.09272925875834%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E99fa4525-d91c-41ea-bcfc-a971b0b8ae0c%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22274.89626562404305%22%20y1%3D%2297.44929927008413%22%20x2%3D%22276.7756607901902%22%20y2%3D%2281.96136037805137%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E1a76128b-a357-4175-bde9-ec966e7b03f4%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22368.9157832822891%22%20y1%3D%22156.66812096862787%22%20x2%3D%22364.8614888161891%22%20y2%3D%22141.72820916441128%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eb3ce60d0-f713-4b32-a245-3c4e49dc661e%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22450.2672083765375%22%20y1%3D%22137.47964263807032%22%20x2%3D%22458.46581311031343%22%20y2%3D%22150.75029118762617%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E1e169e34-7c04-444a-a150-d929699d4c46%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22402.8379615708331%22%20y1%3D%22225.04323260754242%22%20x2%3D%22416.6895416610305%22%20y2%3D%22231.6321440568051%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3E7d1ffc0e-49d3-427a-a002-0cca66c75bff%3C%2Ftitle%3E%3C%2Fline%3E%3Cline%20class%3D%22link%20label_link%22%20x1%3D%22316.39399554833847%22%20y1%3D%22317.34088856556275%22%20x2%3D%22312.1861688457699%22%20y2%3D%22332.00378011188803%22%20link_type%3D%22label_link%22%20pointer-events%3D%22all%22%3E%3Ctitle%3Eda638e42-153b-4dff-ad04-a177b62eccd1%3C%2Ftitle%3E%3C%2Fline%3E%3C%2Fg%3E%3Cg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28261.33022977467175%2C205.7179569780551%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A1%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A1%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28269.92134485342103%2C193.28922349547497%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A2%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A2%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28280.8035276815519%2C182.82036160752085%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A3%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A3%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28293.74361715181374%2C174.6721829685837%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A4%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A4%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28308.00469861783876%2C169.2034001188121%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A5%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A5%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28323.57559352370595%2C167.32372579057125%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A6%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A6%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28318.66952059531167%2C154.31270456052513%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A7%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A7%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28309.7076251862503%2C141.75325952564467%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A8%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A8%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28294.3273054775648%2C143.39093063047775%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A9%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A9%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28280.43262811992%2C137.5041177908933%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A10%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A10%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28271.02504300854974%2C125.95130834466995%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A11%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A11%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28268.3305683413489%2C110.70828701440702%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A12%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A12%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28253.59327465157804%2C102.67471388450429%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A13%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A13%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28237.26378383911214%2C97.76213878304395%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A14%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A14%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28220.20625722162484%2C95.44446768147112%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A15%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A15%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28203.7082151975234%2C90.88999744543656%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A16%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A16%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28188.0665623167126%2C84.49131320760816%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A17%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A17%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28173.88467720906812%2C90.83474235261048%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A18%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A18%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28159.23052399432208%2C87.61903658978987%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A19%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A19%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28148.917393194143%2C76.75091789712536%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A20%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A20%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28146.8605766869757%2C61.38899007091598%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A21%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A21%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28133.2393050741329%2C52.35667415380466%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A22%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A22%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28119.85711929274632%2C43.69077441974384%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A23%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A23%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28107.22978426824206%2C52.28996924736046%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A24%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A24%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2892.64966368977876%2C55.73696961536778%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A25%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A25%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2877.75406024742801%2C53.739059433773846%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A26%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A26%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2864.63373271636493%2C46.51146074728539%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A27%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A27%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2854.98437320541741%2C34.904227706361084%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A28%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A28%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2850.18774085340792%2C20.73495746674428%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A29%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A29%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2850.494542262651024%2C5.371201028906774%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A30%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A30%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2836.0062822565994%2C-1.3295687584877784%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A31%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A31%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2821.37894864564745%2C-7.569605192805647%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A32%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A32%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%286.281139301491906%2C-3.7856753488644146%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A33%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A33%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28-5.825964380613266%2C-12.948879639741032%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A34%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A34%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28-6.166307094561371%2C-28.162600875898775%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A35%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A35%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%285.49192448647406%2C-37.95848201904741%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A36%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A36%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2820.35518375658927%2C-35.05060474965359%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A37%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A37%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2827.51953462907894%2C-21.237525957435718%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A38%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A38%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2841.89663720576248%2C-14.515474357989916%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A39%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A39%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2856.58056588370028%2C-8.294938082925073%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A40%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A40%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2867.61629637228151%2C-18.995330803477085%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A41%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A41%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2881.24485752496938%2C-25.19996419913149%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A42%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A42%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2896.32903750646497%2C-25.995926302116736%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A43%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A43%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28110.54758163432281%2C-21.2571319747247%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A44%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A44%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28122.14546909718675%2C-11.761940409124042%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A45%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A45%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28129.50801838681076%2C1.3943207902431431%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A46%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A46%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28131.5078307762571%2C16.26512688763858%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A47%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A47%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28128.09414019676365%2C31.226922368765038%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A48%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A48%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28141.21854618260056%2C40.350886846244784%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A49%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A49%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28154.62161036492927%2C48.75956934937379%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A50%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A50%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28168.42593069350872%2C42.42212640371588%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A51%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A51%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28183.02865017470933%2C45.09736448038358%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A52%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A52%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28193.10497146266545%2C56.20934898562181%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A53%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A53%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28194.64823371633932%2C71.22539090186221%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A54%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A54%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28208.51615300181905%2C77.22113265498095%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A55%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A55%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28223.07197284879553%2C81.28316697289921%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A56%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A56%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28237.8401227166589%2C83.0285421787688%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A57%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A57%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28252.15477644938574%2C77.84775088217394%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A58%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A58%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28261.3567586734708%2C90.22984782122066%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A59%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A59%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28274.89626562404305%2C97.44929927008413%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A60%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A60%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28286.96624396208074%2C88.23171697410628%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A61%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A61%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28301.6100541832667%2C85.72408910839023%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A62%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A62%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28315.5343057787833%2C91.13604854450232%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A63%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A63%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28324.52450555189927%2C103.32895983867078%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A64%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A64%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28326.1950276616631%2C118.00941509813478%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20rgb%28219%2C%20219%2C%20141%29%3B%22%3E%3Ctitle%3Emolecule_name%3A65%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A65%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28321.4174514972199%2C132.74189680087258%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A66%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A66%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28330.9383778512908%2C147.66942928102875%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A67%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A67%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28338.08486636273875%2C165.38440186906934%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A68%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A68%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28354.76852217911056%2C169.3251818417209%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A69%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A69%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28368.9157832822891%2C156.66812096862787%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A70%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A70%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28384.01925699601077%2C147.58265387332764%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A71%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A71%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28399.4126976058369%2C140.10811503343274%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A72%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A72%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28415.02716627361883%2C133.67140771309843%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A73%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A73%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28430.69891142655683%2C128.1586182221843%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A74%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A74%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28446.03379668066964%2C123.26470642992595%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A75%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A75%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28456.75485643831473%2C112.13331778603275%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A76%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A76%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28471.27646948804073%2C115.7851490720496%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A77%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A77%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28475.4850785880179%2C130.13922480525764%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A78%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A78%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28465.3702123089509%2C141.00669016189443%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightblue%3B%22%3E%3Ctitle%3Emolecule_name%3A79%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A79%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28450.2672083765375%2C137.47964263807032%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A80%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A80%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28435.13499930476445%2C141.91435319605782%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A81%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A81%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28420.05052416224527%2C147.0796197115491%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A82%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A82%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28405.1769335846781%2C153.10120169467294%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A83%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EU%3Ctitle%3Emolecule_name%3A83%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28390.65318886309734%2C160.06071408366773%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A84%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A84%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28377.15880549612746%2C168.11355191907828%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A85%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A85%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28366.8910744841675%2C177.6180403333029%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightgreen%3B%22%3E%3Ctitle%3Emolecule_name%3A86%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A86%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28379.8635256716172%2C186.3614092489082%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A87%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A87%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28390.04179047506284%2C197.62478748408518%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A88%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A88%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28397.86568404176774%2C210.75281579668848%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A89%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A89%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28402.8379615708331%2C225.04323260754242%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A90%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A90%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28404.8591390214511%2C239.9388617360107%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A91%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EG%3Ctitle%3Emolecule_name%3A91%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28404.0257700986887%2C254.89230219200843%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A92%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A92%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28400.2084621543771%2C269.388240769003%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A93%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A93%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28393.60236482910176%2C282.7906104059658%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A94%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A94%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28384.5341062238146%2C294.72053130967106%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A95%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A95%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28373.3472274621469%2C304.68393590894107%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A96%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A96%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28360.4011347697722%2C312.31147081991884%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A97%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A97%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28346.24857705373563%2C317.2339439428048%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A98%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A98%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28331.3507385824045%2C319.69363260866135%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A99%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A99%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28316.39399554833847%2C317.34088856556275%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A100%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A100%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28301.9559657413121%2C313.47119773208703%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A101%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A101%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28287.810092752428%2C308.1462399834821%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A102%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A102%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28276.0605843669028%2C298.65887104079934%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A103%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A103%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28266.20438425261375%2C287.38633987887266%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A104%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A104%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28258.6842665238248%2C274.3830007132106%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A105%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A105%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28253.85197554987042%2C260.185323691355%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A106%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EA%3Ctitle%3Emolecule_name%3A106%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28251.8589387621977%2C245.31339242721728%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%226%22%20node_type%3D%22nucleotide%22%20style%3D%22fill%3A%20lightsalmon%3B%22%3E%3Ctitle%3Emolecule_name%3A107%3C%2Ftitle%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22nucleotide%22%3EC%3Ctitle%3Emolecule_name%3A107%3C%2Ftitle%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28270.1471072104067%2C149.01711800020846%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E10%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28140.5233287677928%2C89.78973909361002%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E20%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2839.21335174637153%2C16.04793652179012%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E30%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%2856.68883556563214%2C-23.818584916003246%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E40%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28159.8504411174667%2C34.09272925875834%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E50%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28276.7756607901902%2C81.96136037805137%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E60%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28364.8614888161891%2C141.72820916441128%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E70%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28458.46581311031343%2C150.75029118762617%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E80%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28416.6895416610305%2C231.6321440568051%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E90%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20struct_name%3D%22molecule_name%22%20transform%3D%22translate%28312.1861688457699%2C332.00378011188803%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%20label%22%20r%3D%226%22%20node_type%3D%22label%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22label%22%3E100%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28143.94542559268615%2C50.859910450784234%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28130.46883903989163%2C41.838916019484415%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%2846.38313559227985%2C-4.606190053823066%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%2831.602907959490235%2C-11.181621809054514%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28328.7494369911492%2C158.33945526641432%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28320.524411996622%2C143.6952103654155%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28264.4243144560423%2C100.53897762538821%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28366.25090062028187%2C167.29294533419386%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28379.9609445573874%2C157.70721458726732%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28394.7012680434687%2C149.94676242840976%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28409.84581135563917%2C143.24342756257%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28425.1654488705516%2C137.47655124647503%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28440.50833520863773%2C132.58044189550117%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28229.5816302692828%2C89.98362226342594%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28213.6825866947351%2C86.73993665995464%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28198.53315096409813%2C81.38187614647559%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28252.7303919958126%2C230.4133669871262%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28256.5577232181126%2C215.93183167292153%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28328.4206869081245%2C242.54626246806174%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%289.995436118288612%2C-20.94542647759887%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28460.7479846776054%2C126.67289815145237%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%2890.64629326415982%2C14.704497093802587%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28297.87530697219375%2C114.85673669316881%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28248.29726996757154%2C90.7454367267056%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3Cg%20class%3D%22noselect%20gnode%22%20transform%3D%22translate%28170.97263402697283%2C66.57020923050078%29%22%20r%3D%226.5%22%3E%3Ccircle%20class%3D%22node%22%20r%3D%220%22%20node_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Fcircle%3E%3Ctext%20text-anchor%3D%22middle%22%20font-size%3D%228%22%20font-weight%3D%22bold%22%20y%3D%222.5%22%20class%3D%22node-label%22%20label_type%3D%22middle%22%3E%3Ctitle%2F%3E%3C%2Ftext%3E%3C%2Fg%3E%3C%2Fg%3E%3Cline%20class%3D%22drag_line%22%20x1%3D%220%22%20y1%3D%220%22%20x2%3D%220%22%20y2%3D%220%22%2F%3E%3C%2Fg%3E%3C%2Fg%3E%3C%2Fsvg%3E"/>

This is one of the sequences from the validation set. Let's start plotting the predicted reactivity and compare to the actual reactivity for the simplest model and work our way up. This picture was generated by the `forna` server that ViennaRNA has made available here: http://rna.tbi.univie.ac.at/forna/

### Defining a useful plotting function for RNA sequences

First, let's define a function to plot the sequences of the predictions versus the actuals, to help us identify the peaks and valleys of our predictions across the sequences.

In [ ]:
def plot_RNA_seq(model, data, idx, figsize = (24,5), grid=True, title=''):
    
    output = model.output    
    fig, ax = plt.subplots(figsize = figsize)        
    
    sub_seq = data.query(f'RNA_idx == {idx}')

    prediction = model.predict(sub_seq)
    ax.plot(range(len(sub_seq)), sub_seq[output], label = 'actuals')
    ax.plot(range(len(sub_seq)), prediction, label = 'pred')

    x_axis1 = list(sub_seq['loop'].values)
    x_axis2 = list(sub_seq['base'].values)

    x_axis = list(zip(x_axis1, x_axis2))
    x_axis = ["_".join(x_axis[i]) for i in range(len(x_axis))]

    ax.set_xticks(range(len(sub_seq)))
    ax.set_xticklabels(x_axis)
    ax.tick_params(rotation = 90)
    ax.set_title(title+str(idx))

    ax.set_ylabel(output)
    ax.set_xlabel('sequence (loop and base)')

    ax.legend()

    if grid:
        ax.grid()

    return ax

In [ ]:
plot_RNA_seq(model_two_features, valid, [642])
display(model_two_features)

This sequence is already captured really well with the simplest, two-feature model of the `loop` and `base_right_motif`. Let's compare it with the three-feature one, that includes the base.

In [ ]:
ax = plot_RNA_seq(model_three_features, valid, [642])
ax.annotate('Region 1', (24,0.8), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 2', (38,1), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 3', (34,0.05), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
display(model_three_features)

It's not the steepest difference, but we see a bit more of a defined peak at the I_A in **region 2** on the plot. We also see a bit more reactivity in the S_G-S_A **region 1**. The I_A-I_C in **region 3** is also much more closely mapped in this prediction.

In general we might see a bit more correction on the stems with the base included.

In [ ]:
ax = plot_RNA_seq(model_constrained, valid, [642])
ax.annotate('Region 1', (24,0.8), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 2', (38,1), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 3', (34,0.05), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 4', (52,1.2), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
display(model_constrained)

With the constrained, four feature model we're getting a little closer to some of the reactive stems in **region 1**, but it comes at the cost of some oversensitivity in other regions. However, looking at our three regions from before, they are now all generally better captured.
We also have a better adjustment for a new region - **region 4**.

This is where it'd make sense to ask yourself what you're trying to accomplish with the model and whether to focus more on performance on specific things, or to understand the dynamics using simpler models.

In [ ]:
ax = plot_RNA_seq(model_base, valid, [642])
ax.annotate('Region 1', (24,0.8), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 2', (38,1), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 3', (34,0.05), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
ax.annotate('Region 4', (52,1.2), bbox=dict(boxstyle="round,pad=0.3", fc='#FFE10A', lw=2))
display(model_base)

As we discovered, having these extra features doesn't really capture much more or add more to the story - it's just curve fitting at this point, trying to squeeze out every last bit of signal. You might see some better adjustments here and there, and some worse performance in other places, but nothing major like what we've seen previously.

## Plotting the development of the RMSE from more complex to simpler models

In [ ]:
print(f'RMSE model features: {len(model_base.features)}: {model_base.rmse(valid):.4f}')
print(f'RMSE model features: {len(model_constrained.features)}: {model_constrained.rmse(valid):.4f}')
print(f'RMSE model features: {len(model_three_features.features)}: {model_three_features.rmse(valid):.4f}')
print(f'RMSE model features: {len(model_two_features.features)}: {model_two_features.rmse(valid):.4f}')

Looking at this, depending on whether you want interpretability or performance, you'd be well off picking between the three or four feature graph.

## Let's take a look at how they perform on the holdout set

In [ ]:
print(f'RMSE model features: {len(model_base.features)}: {model_base.rmse(holdout):.4f}')
print(f'RMSE model features: {len(model_constrained.features)}: {model_constrained.rmse(holdout):.4f}')
print(f'RMSE model features: {len(model_three_features.features)}: {model_three_features.rmse(holdout):.4f}')
print(f'RMSE model features: {len(model_two_features.features)}: {model_two_features.rmse(holdout):.4f}')

We see a good generalization to the holdout set, and a similar story as to which complexity level to pick. This is a very good sign, and shows us that we're on the right track to a possible model on how the structure of an RNA sequence combined with the bases impacts the `reactivity` at each point.

## Concluding remarks

In this example we showed how the `QLattice` can be used to produce simple models that pick out the important features describing `reactivity`. This gives us a better understanding of the mechanisms underlying mRNA reactivity and, because of it's simplicity, is able to generalise to unseen data sets. This is the power of the `QLattice`, simple and explainable models with high predictive performance on unseen data sets!

[Credits](https://docs.abzu.ai/docs/tutorials/python/covid_mrna.html)
